In [1]:
import form
import re
f = form.open("form -l",keep_log=100)

In [2]:
k = [[" 1 ", " 1 ", " 1 ", " 1 "], 
     [" 1 ", " 1 ", " i_", "-i_"], 
     [" 1 ", "-i_", " 1 ", " i_"], 
     [" 1 ", " i_", "-i_", " 1 "]]


n = [[" 1       ", "cx(i1,i2)", "cy(i1,i2)", "cz(i1,i2)"], 
     ["cx(i1,i2)", " 1       ", "cz(i1,i2)", "cy(i1,i2)"], 
     ["cy(i1,i2)", "cz(i1,i2)", " 1       ", "cx(i1,i2)"], 
     ["cz(i1,i2)", "cy(i1,i2)", "cx(i1,i2)", " 1       "]]

c = ["1","cx","cy","cz"]
s = ["1","sx","sy","sz"]
rules = '\n'.join('id ifmatch->endarg '+c[i]+'(i1?,i2?)*'+s[j]+'(i1?,i2?) = '+
                  k[i][j]+'*'+str(n[j][i])+';' 
       for i in range(1,4) for j in range(1,4))
#for i in range(4):
#    print()
#    for j in range(4):
#        print('id m'+str(i)+'*m'+str(j)+' = '\
#              +k[i][j]+'*'+str(n[j][i])+';')
print(rules)

id ifmatch->endarg cx(i1?,i2?)*sx(i1?,i2?) =  1 * 1       ;
id ifmatch->endarg cx(i1?,i2?)*sy(i1?,i2?) =  i_*cz(i1,i2);
id ifmatch->endarg cx(i1?,i2?)*sz(i1?,i2?) = -i_*cy(i1,i2);
id ifmatch->endarg cy(i1?,i2?)*sx(i1?,i2?) = -i_*cz(i1,i2);
id ifmatch->endarg cy(i1?,i2?)*sy(i1?,i2?) =  1 * 1       ;
id ifmatch->endarg cy(i1?,i2?)*sz(i1?,i2?) =  i_*cx(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sx(i1?,i2?) =  i_*cy(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sy(i1?,i2?) = -i_*cx(i1,i2);
id ifmatch->endarg cz(i1?,i2?)*sz(i1?,i2?) =  1 * 1       ;


In [3]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
Fun Comm;
auto sym a,b;

L L = Comm(a1+a2+a3,b1+b2);

*--- expand ---
splitarg Comm,1;
repeat id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
splitarg Comm,2;
repeat id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
.sort
''')
f.read('L')

'Comm(a1,b1)+Comm(a1,b2)+Comm(a2,b1)+Comm(a2,b2)+Comm(a3,b1)+Comm(a3,b2)'

In [4]:
procedures = '''
cfun   cx,cy,cz,cc; *cten
set CC:cx,cy,cz;
nfun   sx,sy,sz,sc;
set SC:sx,sy,sz;

Function S,SS,Comm;

auto Symbols a,b,t;
auto Symbol n,i,j,k; * indices

S n,n1,n2,x;
Table ident(x?);  Fill ident() = x;
Table prod(n?symbol_,n1?int_,n2?int_,x?);
Fill prod() =
  + theta_(n2-n1) * prod(n,n1,n2-1,x) * ident(x * replace_(n,n2))
  + thetap_(n1-n2)
;

S x,y,z;
Table if(x?int_,y?,z?);
Fill if() = deltap_(x)*y+delta_(x)*z;

*--- expand ---
#procedure expand
splitarg Comm,1;
repeat id Comm(a?,a1?,?centr,b?) = Comm(a,b) + Comm(a1,?centr,b);
id Comm(a?,a1?,b?) = Comm(a,b)+Comm(a1,b);
splitarg Comm,2;
repeat id Comm(a?,?centr,b1?,b?) = Comm(a,b) + Comm(a,?centr,b1);
id Comm(a?,b1?,b?) = Comm(a,b)+Comm(a,b1);
#endprocedure

*--- overlapping ---
#procedure overlapping
argument Comm;
    multiply S(1,1,1);
    repeat id S(k1?,k2?,t?)*cc?CC(i1?,i2?) = S(max_(k1,i1),max_(k2,i2),t*cc(i1,i2));
endargument;
id Comm(a?,b?) = a*b;
id S(?ss1)*S(?ss2) = Comm(S(?ss1),S(?ss2));
id Comm(S(n1?,k1?,a1?),S(n2?,k2?,a2?)) = 
                  Comm(S(k1,        a1),S(k2,        a2))  + 
    sum_(i,1,n2-1,Comm(S(k1,SS(i,1)*a1),S(k2,        a2))) +
    sum_(i,1,n1-1,Comm(S(k1,        a1),S(k2,SS(i,1)*a2)));
argument Comm; argument S,2;
    repeat id SS(i?,t?)*cc?CC(j?,k?) = SS(i,t*cc(i+j,k));
    id SS(i?,a?) = a;
endargument;endargument;
id Comm(S(n1?,a1?),S(n2?,a2?)) = 
                  Comm(        a1,        a2) + 
    sum_(i,1,n2-1,Comm(SS(i,1)*a1,        a2)) +
    sum_(i,1,n1-1,Comm(        a1,SS(i,1)*a2));
argument Comm;
    repeat id SS(i?,t?)*cc?CC(k?,j?) = SS(i,t*cc(k,i+j));
    id SS(i?,a?) = a;
endargument;
#endprocedure

*--- multiply ---
#procedure multiply
*id Comm(a?,b?) = S(a)*b-S(b)*a;
label repeat;
    id ifnomatch->endrepeat S(a?)*cc?CC[n](i1?,i2?) = S(a*SC[n](i1,i2));
    argument S;
*        id cc?CC(i1?,i2?)*sc?SC(i1?,i2?) = 1;
'''+rules+\
'''
        id sc?SC[n?](i1?,i2?) = CC[n](i1,i2);
        label endarg;
    endargument;
goto repeat;
label endrepeat;
id S(a?) = a;
*.sort
#endprocedure

*--- trim ---
#procedure trim
if(match(cc?CC(i?,j?)));
    multiply S(1000000,1000000,1);
    repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(min_(n1,i1),min_(n2,i2),t*cc(i1,i2));
    id ifmatch->endif S(1,1,t?) = t;
        id S(n1?,n2?,t?) = S(n1-1,n2-1,1)*t;
        repeat id S(n1?,n2?,t?)*cc?CC(i1?,i2?) = S(n1,n2,t*cc(i1-n1,i2-n2));
        id S(n1?,n2?,t?) = t;
    label endif;
endif;
#endprocedure
'''

In [5]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
'''+procedures+
'''

Local H  = cx(1,1)*cx(1,2) + cy(1,1)*cy(2,1);
Local N1 = cx(1,1)*cx(1,2) - cy(1,1)*cy(2,1);
.sort
Skip;
Local N2 = Comm(H,N1);

#call expand
#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
skip; nskip N2;
#call trim

format mathematica;
.sort
''')
print(re.sub(r'i_','I',f.read('N2')))

-4*cx[1,1]*cy[2,2]*cz[1,2]*I-4*cx[1,2]*cy[2,1]*cz[1,1]*I-4*cx[2,1]*cy[1,2]*cz[2,2]*I-4*cx[2,2]*cy[1,1]*cz[2,1]*I


In [6]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
Sym h,J;

*cfun koef;
*polyfun koef;

'''+procedures+
'''

Table X(n?); Fill X() = cx(1,1)*prod(i,2,n,cz(1,i))*cx(1,n+1);
Table Y(n?); Fill Y() = if(thetap_(n),cy(1,1)*prod(i,2,n,cz(1,i))*cy(1,n+1),-cz(1,1));
Table M(n?); Fill M() = if(thetap_(n),
    (cx(1,1)*prod(i,2,n,cz(1,i))*cy(1,n+1)+cy(1,1)*prod(i,2,n,cz(1,i))*cx(1,n+1))/2,
    0);
Table Im(n?); Fill Im() = if(thetap_(n),
    (cx(1,1)*prod(i,2,n,cz(1,i))*cy(1,n+1)-cy(1,1)*prod(i,2,n,cz(1,i))*cx(1,n+1))/2,
    0);
Table Ip(n); Fill Ip() = h*X(n) + h*Y(n) - X(n+1) - Y(n-1);
.global
G H = -J*cx(1,1)*cx(1,2)-J*h*cz(1,1);
.store

#define N "50"
L HIp1 = Comm(H,Ip(`N'));
L HIm1 = Comm(H,Im(`N'));
L HX1   = i_*Comm(H,X(`N')) - 4*J*h*M(`N') + 4*J*M(`N'-1);
L HY1   = i_*Comm(H,Y(`N')) + 4*J*h*M(`N') - 4*J*M(`N'+1);
L HM1   = i_*Comm(H,M(`N'))  +2*J*h*X(`N') - 2*J*h*Y(`N') - 2*J*X(`N'+1) + 2*J*Y(`N'-1);


#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

.sort
''')

print(f.read('HIp1'))
print(f.read('HIm1'))
print(f.read('HX1'))
print(f.read('HY1'))
print(f.read('HM1'))

0
0
0
0
0


In [8]:
f.close()
f = form.open("form -l",keep_log=100)
f.write('''
on stats;
Sym h,J;

*cfun koef;
*polyfun koef;

'''+procedures+
'''

Table X(n?); Fill X() = cx(1,1)*prod(i,2,n,cz(i,1))*cx(n+1,1);
Table Y(n?); Fill Y() = if(thetap_(n),cy(1,1)*prod(i,2,n,cz(i,1))*cy(n+1,1),-cz(1,1));
Table M(n?); Fill M() = if(thetap_(n),
    (cx(1,1)*prod(i,2,n,cz(i,1))*cy(n+1,1)+cy(1,1)*prod(i,2,n,cz(i,1))*cx(n+1,1))/2,
    0);
Table Im(n?); Fill Im() = if(thetap_(n),
    (cx(1,1)*prod(i,2,n,cz(i,1))*cy(n+1,1)-cy(1,1)*prod(i,2,n,cz(i,1))*cx(n+1,1))/2,
    0);
Table Ip(n); Fill Ip() = h*X(n) + h*Y(n) - X(n+1) - Y(n-1);
.global
G H = -J*cx(1,1)*cx(2,1)-J*h*cz(1,1);
.store

#define N "50"
L HIp1 = Comm(H,Ip(`N'));
L HIm1 = Comm(H,Im(`N'));
L HX1   = i_*Comm(H,X(`N')) - 4*J*h*M(`N') + 4*J*M(`N'-1);
L HY1   = i_*Comm(H,Y(`N')) + 4*J*h*M(`N') - 4*J*M(`N'+1);
L HM1   = i_*Comm(H,M(`N'))  +2*J*h*X(`N') - 2*J*h*Y(`N') - 2*J*X(`N'+1) + 2*J*Y(`N'-1);


#call overlapping
id Comm(a?,b?) = S(a)*b-S(b)*a;
#call multiply
.sort
#call trim

.sort
''')

print(f.read('HIp1'))
print(f.read('HIm1'))
print(f.read('HX1'))
print(f.read('HY1'))
print(f.read('HM1'))

0
0
0
0
0
